In [ ]:
import os, sys
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import torch as t
from torch import nn, Tensor
from torch.distributions.categorical import Categorical
from torch.nn import functional as F
from dataclasses import dataclass
import numpy as np
import einops
from jaxtyping import Float, Int
from typing import Optional, Callable, Union, List, Tuple
from functools import partial
from tqdm.notebook import tqdm
from dataclasses import dataclass
from rich import print as rprint
from rich.table import Table
from IPython.display import display, HTML
from pathlib import Path
import pandas as pd
pd.options.plotting.backend = "plotly"
import matplotlib.pyplot as plt

chapter = "chapter1_transformer_interp"
exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
section_dir = (exercises_dir / "part4_superposition_and_saes").resolve()
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

from plotly_utils import imshow, line, hist
from part31_superposition_and_saes.utils import (
    plot_features_in_2d,
    plot_features_in_Nd,
    plot_features_in_Nd_discrete,
    plot_correlated_features,
    plot_feature_geometry,
    frac_active_line_plot,
)
device = t.device("cuda" if t.cuda.is_available() else "cpu")

if not t.backends.mps.is_available():
    if not t.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    device = t.device("mps")

MAIN = __name__ == "__main__"

In [ ]:
def linear_lr(step, steps):
    return (1 - (step / steps))

def constant_lr(*_):
    return 1.0

def cosine_decay_lr(step, steps):
    return np.cos(0.5 * np.pi * step / (steps - 1))


@dataclass
class Config:
    # We optimize n_instances models in a single training loop to let us sweep over
    # sparsity or importance curves  efficiently. You should treat `n_instances` as
    # kinda like a batch dimension, but one which is built into our training setup.
    n_instances: int
    n_features: int = 5
    n_hidden: int = 2
    n_correlated_pairs: int = 0
    n_anticorrelated_pairs: int = 0


class Model(nn.Module):
    W: Float[Tensor, "n_instances n_hidden n_features"]
    b_final: Float[Tensor, "n_instances n_features"]

    def __init__(
        self,
        cfg: Config,
        feature_probability: Optional[Union[float, Tensor]] = None,
        importance: Optional[Union[float, Tensor]] = None,
        device = device,
    ):
        super().__init__()
        self.cfg = cfg

        if feature_probability is None: feature_probability = t.ones(())
        if isinstance(feature_probability, float): feature_probability = t.tensor(feature_probability)
        self.feature_probability = feature_probability.to(device)
        if importance is None: importance = t.ones(())
        if isinstance(importance, float): importance = t.tensor(importance)
        self.importance = importance.to(device).broadcast_to((cfg.n_instances, cfg.n_features))

        self.W = nn.Parameter(nn.init.xavier_normal_(t.empty((cfg.n_instances, cfg.n_hidden, cfg.n_features))))
        self.b_final = nn.Parameter(t.zeros((cfg.n_instances, cfg.n_features)))
        self.to(device)


    def forward(
        self,
        features: Float[Tensor, "... instances features"]
    ) -> Float[Tensor, "... instances features"]:

        h = einops.einsum(features, self.W, "... instances features, instances n_hidden features -> ... instances n_hidden")
        
        output = einops.einsum(h, self.W, "... instances n_hidden, instances n_hidden features -> ... instances features")
        
        return F.relu(output + self.b_final).to(device)


    def generate_batch(self, batch_size) -> Float[Tensor, "batch_size instances features"]:
        '''
        Generates a batch of data. We'll return to this function later when we apply correlations.
        '''
        features = t.rand((batch_size, self.cfg.n_instances, self.cfg.n_features), device = self.W.device)
    
        feature_seed = t.rand((batch_size, self.cfg.n_instances, self.cfg.n_features), device = self.W.device)
        
        feat_is_present = feature_seed <= self.feature_probability
        
        batch = t.where(feat_is_present, features, 0.0)
        
        return batch


    def calculate_loss(
        self,
        out: Float[Tensor, "batch instances features"],
        batch: Float[Tensor, "batch instances features"],
    ) -> Float[Tensor, ""]:
        '''
        Calculates the loss for a given batch, using this loss described in the Toy Models paper:

            https://transformer-circuits.pub/2022/toy_model/index.html#demonstrating-setup-loss

        Remember, `model.importance` will always have shape (n_instances, n_features).
        '''
        squared_loss = self.importance * (batch - out) ** 2
    
        loss = einops.reduce(squared_loss, "batch instances features -> instances", "mean")
        
        summed_loss = loss.sum()
        
        return summed_loss, loss


    def optimize(
        self,
        batch_size: int = 1024,
        steps: int = 10_000,
        log_freq: int = 100,
        lr: float = 1e-3,
        lr_scale: Callable[[int, int], float] = constant_lr,
    ):
        '''
        Optimizes the model using the given hyperparameters.
        '''
        optimizer = t.optim.Adam(list(self.parameters()), lr=lr)

        progress_bar = tqdm(range(steps))
        
        losses = []
        summed_losses = []

        for step in progress_bar:

            # Update learning rate
            step_lr = lr * lr_scale(step, steps)
            for group in optimizer.param_groups:
                group['lr'] = step_lr

            # Optimize
            optimizer.zero_grad()
            batch = self.generate_batch(batch_size)
            out = self(batch)
            summed_loss, loss = self.calculate_loss(out, batch)
            losses.append(loss.detach().cpu().clone().numpy().tolist())
            summed_losses.append(summed_loss.detach().cpu().clone().item())
            summed_loss.backward()
            optimizer.step()

            # Display progress bar
            if step % log_freq == 0 or (step + 1 == steps):
                progress_bar.set_postfix(loss=summed_loss.item()/self.cfg.n_instances, lr=step_lr)
                
        return summed_losses,losses

In [ ]:
def feature_probability(n_instances):
    feature_prob = (50 ** -t.linspace(0, 1, n_instances))
    feature_prob = einops.rearrange(feature_prob, "instances -> instances ()")
    return feature_prob

def feature_importance(n_features):
    importance = (0.7 ** t.arange(n_features))
    importance = einops.rearrange(importance, "features -> () features")
    return importance
    

In [ ]:
cfg = Config(
    n_instances = 8,
    n_features = 8,
    n_hidden = 2,
)

# importance varies within features for each instance
importance = feature_importance(cfg.n_features)

# sparsity is the same for all features in a given instance, but varies over instances
feature_prob = feature_probability(cfg.n_instances)

In [ ]:
importance

In [ ]:
importance

In [ ]:
model = Model(
    cfg = cfg,
    device = device,
    importance = importance,
    feature_probability = feature_prob,
)
summed_losses, losses = model.optimize(steps=10_000)

In [ ]:
loss_df = pd.DataFrame(losses, columns=[str(np.round(prob.item(),3)) for prob in feature_prob])
loss_df["total_loss"] = loss_df.apply(lambda x: x.sum(),axis = 1)

In [ ]:
fig = np.log(loss_df).plot(title = f"Loss Curves: {cfg.n_features} features represented in 2D space", height=600).update_layout(xaxis_title="Steps", yaxis_title="Log loss")
fig.write_image("higher_input_feature_plots/random_test.png")

In [ ]:
np.log(loss_df).shape

In [ ]:
loss_df.columns.name = 'Sparsity'
np.log(loss_df).plot(title = f"Loss Curves: {cfg.n_features} features represented in 2D space", height=600).update_layout(
    xaxis_title="Steps", yaxis_title="Log loss"
).show()

In [ ]:
plt.matshow(t.corrcoef(model.W.detach()[-1].T).cpu().numpy())

In [ ]:
model.W.detach()[-1].T, t.corrcoef(model.W.detach()[-1].T)

In [ ]:
model.W.detach()

In [ ]:
model.importance

In [ ]:
plot_features_in_2d(
    model.W.detach(),
    colors = model.importance,
    title = f"Superposition: {cfg.n_features} features represented in 2D space",
    subplot_titles = [f"1 - S = {i:.3f}" for i in feature_prob.squeeze()],
)

In [ ]:
def input_feature_plots(n_feature):
    print(f"n_feature:{n_feature}")

    cfg = Config(
        n_instances = 8,
        n_features = n_feature,
        n_hidden = 2,
    )
    
    # importance varies within features for each instance
    importance = feature_importance(cfg.n_features)
    
    # sparsity is the same for all features in a given instance, but varies over instances
    feature_prob = feature_probability(cfg.n_instances)

    model = Model(
        cfg = cfg,
        device = device,
        importance = importance,
        feature_probability = feature_prob,
    )
    summed_losses, losses = model.optimize(steps=10_000)

    loss_df = pd.DataFrame(losses, columns=[np.round(prob.item(),3) for prob in feature_prob])
    loss_df["total_loss"] = loss_df.apply(lambda x: x.sum(),axis = 1)
    
    loss_df.columns.name = 'Sparsity'
    
    np.log(loss_df).plot(title = f"Loss Curves: {cfg.n_features} features represented in 2D space", height=600).update_layout(xaxis_title="Steps", yaxis_title="Log loss").show()

    loss_df.plot(title = f"Loss Curves: {cfg.n_features} features represented in 2D space", height=600).update_layout(xaxis_title="Steps", yaxis_title="Loss").show()

    plot_features_in_2d(
        model.W.detach(),
        colors = model.importance,
        title = f"Superposition: {cfg.n_features} features represented in 2D space",
        subplot_titles = [f"1 - S = {i:.3f}" for i in feature_prob.squeeze()],
    )
                

In [ ]:
def different_input_features_runs(n_features):
    print(f"n_features:{n_features}")

    cfg = Config(
            n_instances = 8,
            n_features = n_features,
            n_hidden = 2,
        )

    
    # importance varies within features for each instance
    importance = feature_importance(cfg.n_features)
    
    # sparsity is the same for all features in a given instance, but varies over instances
    feature_prob = feature_probability(cfg.n_instances)

    model = Model(
        cfg = cfg,
        device = device,
        importance = importance,
        feature_probability = feature_prob,
    )
    summed_losses, losses = model.optimize(steps=10_000)

    results_dict = {}
    results_dict["W"] = model.W.detach()
    results_dict["b"] = model.b_final.detach()
    results_dict["summed_loss"] = summed_losses
    results_dict["losses"] = losses
    
    return results_dict

In [ ]:
n_features_list = list(range(2,11))
results_list = []

for n_feature in n_features_list:
    results_list.append(different_input_features_runs(n_feature))

In [ ]:
def avg_frob_norm(W):
    return np.mean([t.norm(w).item() for w in W])

def max_frob_norm(W):
    return np.max([t.norm(w).item() for w in W])

In [ ]:
for i in range(len(n_features_list)):
    print(f"n_feature: {n_features_list[i]}")
    W = results_list[i]["W"].cpu()
    print(f"avg norm: {avg_frob_norm(W)}")
    print(f"max norm: {max_frob_norm(W)}")

In [ ]:
n_feature_list = list(range(2,11))
input_feature_plots(2)

In [ ]:
input_feature_plots(3)

In [ ]:
input_feature_plots(4)

In [ ]:
input_feature_plots(5)

In [ ]:
input_feature_plots(6)

In [ ]:
input_feature_plots(7)

In [ ]:
input_feature_plots(8)

In [ ]:
input_feature_plots(9)

In [ ]:
input_feature_plots(10)